# Crawling [bezdim.org](https://bezdim.org/signali/reports)

### Import Scrapy and install if missing

In [1]:
try:
    import scrapy
except:
    import sys
    !conda install --yes --prefix {sys.prefix} -c conda-forge scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
import json
import codecs

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = codecs.open('reportresult.jl', 'w+', 'utf-16')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [3]:
import logging

class BezDimSpider(scrapy.Spider):
    name = "bezdim"
    start_urls = [
        'https://bezdim.org/signali/reports'
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEED_FORMAT':'json',
        'FEED_URI': 'reports-result.json'
    }
    
    def parse(self, response):
        for report in response.css('div.rb_report'):
            title = report.css('a.r_title::text').extract_first().strip()
            description = report.css('div.r_description::text').extract_first().strip()
            address = report.css('p.bd_location::text').extract_first().strip()
            
            print("Title: ", title, "Adress: ", address)
            
            yield {
                'title': title,
                'description': description,
                'address': address
            }

In [4]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(BezDimSpider)
process.start()

2019-01-24 20:54:44 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2019-01-24 20:54:44 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 18.9.0, Python 3.7.0 (default, Jun 28 2018, 07:39:16) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 18.0.0 (OpenSSL 1.0.2p  14 Aug 2018), cryptography 2.3.1, Platform Darwin-18.0.0-x86_64-i386-64bit
2019-01-24 20:54:44 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'reports-result.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


Title:  Тютюнопушене в ресторант "Вестител", град Враца Adress:  град Враца, ул. "Вестителя" № 22
Title:  Пушене в ресторант "РЕАНА", град Варна (м) Adress:  Гград Варна, ул. "Братя Бъкстон" № 35
Title:  Пушене в ресторант "Елеа", град Пловдив (м) Adress:  град Пловдив, бул. Санкт Петербург 38, Лаута
Title:  НЕ СПАЗВАНЕ ПРАВИЛАТА ЗА ПУШЕНЕ В ЗАВЕДЕНИЕ СМОЦИТЕ В СЕЛО СВЕТОВРАЧАНЕ, ГРАД СОФИЯ Adress:  село Световрачане, Столична община
Title:  Пушене в заведение Гардън Бар, град Варна Adress:  град Варна,  ул."Отец Паисий"№ 22
Title:  Пушене в заведение Nebo, град София Adress:  град София, ул Шипка 34
Title:  Нарушение на забраната за тютюнопушене в заведение "Макаби", град Асеновград Adress:  град Асеновград, бул. България 12
Title:  Пушене в бистро "Sweet cherry bar", град Бургас (м) Adress:  град Бургас, Славейков бл. 86
Title:  Пушене в кафе-аперитив и магазин "РИКО-33 ЕООД", село Ракита, област Плевен Adress:  село Ракита, област Плевен, ул. Георги Димитров № 7
Title:  Пушене в мех